In [12]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [13]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "..\\chrome-win64\\chrome.exe" # chrome 경로 설정
options.add_argument("--no-sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage") # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # 화면크기설정, 반응형 웹 대비
# options.add_argument("--disable-gpu") # GPU 비활성화

# ChromeDriver 설정
service = Service("..\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.pknu.ac.kr/main/399")
print(driver.title)

menu = driver.find_element(By.CSS_SELECTOR, "#subCont > table > tbody > tr:nth-child(1) > td.bdlTitle > a").click()
time.sleep(5)
menu_option = driver.find_element(By.CSS_SELECTOR, "#container")
time.sleep(2)
html = driver.page_source

교내 식당 주간식단표


In [24]:
soup = bs(html,"html.parser")
day = []
day2 = []
menu = []
# 본문내용 가져오기
for a in range(2,7):
    contents = soup.select(f"#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(1) > td:nth-child({a})")
    for c in contents:
        a = c.get_text()
        day.append(a)
print(day)

for a in range(1,6):
    contents = soup.select(f"#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(2) > td:nth-child({a})")
    for c in contents:
        a = c.get_text()
        day2.append(a)
print(day2)

for i in range(2,7):
    for a in range(1,11):
        contents = soup.select(f"#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(3) > td:nth-child({i}) > p:nth-child({a}) ")
        for c in contents:
            a = c.get_text()
        menu.append(a)
print(menu)

print(len(menu))



['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
['4월 14일', '4월 15일', '4월 16일', '4월 17일', '4월 18일']
['흑미밥/현미밥', '콩나물김치국', '불닭볶음', '고구마맛탕', '잡채어묵조림', '청포묵김무침', '그린샐러드', '키위/오리엔탈드레싱', '푸실리오일샐러드', '배추김치', '흑미밥/현미밥', '순두부찌개', '몽골리안돈까스', '순대볶음', '우엉조림', '배추찜', '그린샐러드', '흑임자/발사믹드레싱', '콘슬로우샐러드', '깍두기', '흑미밥/현미밥', '닭개장', '갈릭마요미트볼', '두부조림', '들기름막국수', '시금치나물', '그린샐러드', '석류/오리엔탈드레싱', '펜네샐러드', '베추김치', ' 흑미밥/현미밥', '시락국', '돈육분짜무침', '야채고로케/케찹', '콩나물무침', '고추지무침', '그린샐러드', '참깨/오렌지드레싱', '마카로니샐러드', '깍두기', '현미밥', '목살필라프', '달걀국', '돈육맛쵸킹', '짜장소스', '중국식오이무침', '그린샐러드', '딸기/파인드레싱', '실곤약샐러드', '배추김치']
50


In [23]:
soup = bs(html, 'html.parser')

# 요일 및 날짜 수집
days = [c.get_text().strip() for c in soup.select("#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(1) > td")]
dates = [c.get_text().strip() for c in soup.select("#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(2) > td")]

# 전체 메뉴 수집 (월~금: td 2~6번)
menu_data = []
for col in range(2, 7):  # 2~6번째 td (월~금)
    contents = [c.get_text().strip() for c in soup.select(
        f"#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(3) > td:nth-child({col})")]
    menu_data.append(contents)

# 리스트 길이 확인 및 DataFrame 생성
if len(days) == len(dates) == len(menu_data):
    df = pd.DataFrame({
        '요일': days,
        '날짜': dates,
        '메뉴': menu_data
    })
    print(df.to_string(index=False))
df

,요일,날짜,메뉴
0,Monday,4월 14일,[흑미밥/현미밥콩나물김치국불닭볶음고구마맛탕잡채어묵조림청포묵김무침그린샐러드키위/오리엔...
1,Tuesday,4월 15일,[흑미밥/현미밥순두부찌개몽골리안돈까스순대볶음우엉조림배추찜그린샐러드흑임자/발사믹드레싱...
2,Wednesday,4월 16일,[흑미밥/현미밥닭개장갈릭마요미트볼두부조림들기름막국수시금치나물그린샐러드석류/오리엔탈드...
3,Thursday,4월 17일,[ 흑미밥/현미밥시락국돈육분짜무침야채고로케/케찹콩나물무침고추지무침그린샐러드참깨/오렌...
4,Friday,4월 18일,[현미밥목살필라프달걀국돈육맛쵸킹짜장소스중국식오이무침그린샐러드딸기/파인드레싱실곤약샐러...


In [5]:
# soup = bs(html, 'html.parser')
# contents = soup.select("#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18)")

# for c in contents:
#     menu = c.get_text(strip=True).split("\n")
# c

In [6]:
#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table > tbody > tr:nth-child(3) > td:nth-child(1)

In [ ]:
#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > tablld(2)